In [3]:
import torch 
import torch.nn as nn 
class CausalAttention(nn.Module):
    def __init__(self, dim_in, dim_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.W_key = nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.W_value = nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, _ = x.shape
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        attention_scores = queries @ keys.transpose(1,2) 
        masked_scores = attention_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attention_weights = torch.softmax(masked_scores/keys.shape[-1] ** 0.5, dim=-1)
        attention_weights = self.dropout(attention_weights)

        context_vec = attention_weights @ values
        return context_vec
        

In [4]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self,  dim_in, dim_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = [CausalAttention(dim_in, dim_out, context_length, dropout, qkv_bias) for _ in range(num_heads)]

    def forward(self, x):
        return torch.cat( [head(x) for head in self.heads], dim = -1)
        

In [5]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [6]:
batch = torch.stack((inputs, inputs), dim=0)
batch.shape

torch.Size([2, 6, 3])

In [7]:
context_len = batch.shape[1]
dim_in = 3 
dim_out = 2
mha = MultiHeadAttentionWrapper(dim_in, dim_out, context_len, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)

tensor([[[-0.3760,  0.0589, -0.1819, -0.4972],
         [-0.3896,  0.1173, -0.2070, -0.5986],
         [-0.3924,  0.1363, -0.2164, -0.6300],
         [-0.3464,  0.1285, -0.1885, -0.5650],
         [-0.3144,  0.1223, -0.1980, -0.5347],
         [-0.3086,  0.1230, -0.1774, -0.5195]],

        [[-0.3760,  0.0589, -0.1819, -0.4972],
         [-0.3896,  0.1173, -0.2070, -0.5986],
         [-0.3924,  0.1363, -0.2164, -0.6300],
         [-0.3464,  0.1285, -0.1885, -0.5650],
         [-0.3144,  0.1223, -0.1980, -0.5347],
         [-0.3086,  0.1230, -0.1774, -0.5195]]], grad_fn=<CatBackward0>)


### Implementing MultiHeadAttention with weight splits. 

In [22]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_len, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert(d_out % num_heads == 0), \
             "d_out must be divisible by num_heads"
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = self.d_out//self.num_heads

        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out) # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x) # (b, num_tokens, d_out)

        #now d_out dim will be rolledout into num_heads, head_dim
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        #Currenlty this is how these matrices look : (b, num_tokens, num_heads, head_dim). 
        #Going ahead we will have to switch num_heads <-> num_token, (Group by num_heads) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1,2)
        queries = queries.transpose(1,2)
        values = values.transpose(1,2)
        #next step : Finding the attention scores
        attention_scores = queries @ keys.transpose(2,3)
        #next step: calculating attention weights.
        # - apply the mask , divide be sqrt(head_dim), apply softmax
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attention_scores.masked_fill_(mask_bool, -torch.inf)
        attention_weights = torch.softmax(attention_scores / keys.shape[-1] ** 0.5, dim =-1)
        # apply droputs
        attention_weights = self.dropout(attention_weights)
        # shape of the attention weights (b, num_heads, num_tokens, num_tokens)
        # to find the context_vectors we will multiply attention_scores with values matrx (b, num_heads, num_tokens, head_dim)
        # so the output wouldbe (b, num_heads, num_tokens, head_dim).
        # we will also swap back the num_heads with num_tokens -> (b, num_tokens, num_heads, head_dim)
        context_vectors = (attention_weights @ values).transpose(1,2)

        # now its time to roll back in the num_heads and head_dim into d_out. 
        context_vectors = context_vectors.contiguous().view(b, num_tokens, self.d_out)
        context_vectors = self.out_proj(context_vectors)

        return context_vectors
            

In [23]:
torch.manual_seed(123)

# Define the tensor with 3 rows and 6 columns
inputs = torch.tensor(
    [[0.43, 0.15, 0.89, 0.55, 0.87, 0.66],  # Row 1
     [0.57, 0.85, 0.64, 0.22, 0.58, 0.33],  # Row 2
     [0.77, 0.25, 0.10, 0.05, 0.80, 0.55]]  # Row 3
)

batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) 

batch_size, context_length, d_in = batch.shape
d_out = 6
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

torch.Size([2, 3, 6])
tensor([[[ 0.1569, -0.0873,  0.0210,  0.0215, -0.3243, -0.2518],
         [ 0.1125, -0.0561,  0.0454, -0.0234, -0.3247, -0.3030],
         [ 0.1194, -0.0501,  0.0347, -0.0637, -0.2797, -0.2612]],

        [[ 0.1569, -0.0873,  0.0210,  0.0215, -0.3243, -0.2518],
         [ 0.1125, -0.0561,  0.0454, -0.0234, -0.3247, -0.3030],
         [ 0.1194, -0.0501,  0.0347, -0.0637, -0.2797, -0.2612]]],
       grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 3, 6])
